# Two port electrical parameters
For two port electrical systems there are a family of related parameters to characterize the system. For radio-frequency applications s-parameters are a common choice to indicate the electrical response of the system. This family of electrical parameters are chosen based on what type of analysis is required, they effectively encode much of the same information so that they may be interchanged in many instances. 
1. S-parameters
2. Z-parameters
3. Y-parameters
4. T-parameters
5. ABCD-parameters
6. H-parameters

## Creating a graph that connects the parameters. 
As can be seen a set of parameters may be chosen based on the analysis needed, by creating a "content" graph we can have any set of parameters given one set. To do this we must first determine which information is required.

## Basic data model
Each of the parameters for the two-port can be represented by a matrix, each of these matrix representations are frequency dependent. This lends it self to a data model that is a python list [[fi,m11,m12,m21,m22],...  or a list of matrices with a frequency list. [[f,matrix()]..], if a matrix determinant of inverse is required then use matrix form otherwise use the list form. The port and load impedance must be specified to use all of the transforms and frequency units should be specified.

In [7]:
import numpy as np
from pyMeasure import *
from pyMeasure.Code.DataHandlers.GraphModels import *

Importing Code.Utils.Names
Importing Code.DataHandlers.NISTModels
Importing Code.DataHandlers.GeneralModels
Importing Code.DataHandlers.TouchstoneModels
Importing Code.DataHandlers.XMLModels
Importing Code.DataHandlers.RadiCALModels
Importing Code.DataHandlers.ZipModels
Importing Code.DataHandlers.Translations
Importing Code.DataHandlers.StatistiCALModels
Importing Code.DataHandlers.MUFModels
Importing Code.Analysis.SParameter
Importing Code.InstrumentControl.Instruments
Importing Code.InstrumentControl.Experiments


In [66]:
# first we define matrix to list conversions
def FrequencyList_to_FrequencyMatrixList(frequency_list):
    """Converts a list of form [f,m11,m12,m21,m22] to a list of form [f,np.matrix([[m11,m12],[m12,m22]])]
    inverse of FrequencyMatrixList_to_FrequencyList """
    matrix_list=[]
    for row in frequency_list[:]:
        frequency=row[0]
        [m11,m12,m21,m22]=row[1:]
        matrix=np.matrix([[m11,m12],[m21,m22]])
        matrix_list.append([frequency,matrix])
    return matrix_list

def FrequencyMatrixList_to_FrequencyList(frequency_matrix_list):
    """Converts a list of form [f,np.matrix([[m11,m12],[m12,m22]])] to a list of form [f,m11,m12,m21,m22]
    inverse of FrequencyList_to_FrequencyMatrixList"""
    frequency_list=[]
    for row in frequency_matrix_list[:]:
        frequency=row[0]
        matrix=row[1]
        [m11,m12,m21,m22]=[matrix[0,0],matrix[1,0],matrix[0,1],matrix[1,1]]
        frequency_list.append([frequency,m11,m12,m21,m22])
    return frequency_list

# now for each set of parameters we need a single conversion into and out of the format

# This was copied and needs to be refactored to be consitent with the other tranforms
def SFrequencyMatrixList_to_TFrequencyMatrixList(S_list):
    """Converts S-parameters into a T Matrix. Input form should be in frequency, np.matrix([[S11,S12],[S21,S22]])
    format. Returns a list in [frequency, np.matrix] format.
    Is the inverse of TFrequencyMatrixList_to_SFrequencyMatrixList"""
    t_complex_list=[]
    t_matrix=[]
    for row in S_list:
        frequency=row[0]
        m=row[1]
        T11=-np.linalg.det(m)/m[1,0]
        T12=m[0,0]/m[1,0]
        T21=-m[1,1]/m[1,0]
        T22=1/m[1,0]
        t_matrix.append([frequency,np.matrix([[T11,T12],[T21,T22]])])
        t_complex_list.append([frequency,T11,T12,T21,T22])
    return t_matrix

def TFrequencyMatrixList_to_SFrequencyMatrixList(T_list):
    """Converts T Matrix into S parameters. Input form should be in frequency, np.matrix([[T11,T12],[T21,T22]])
    format. Returns a list in [frequency, np.matrix] format. 
    Is the inverse of SFrequencyMatrixList_to_TFrequencyMatrixList"""
    S_list=[]
    for row in T_list:
        frequency=row[0]
        m=row[1]
        S11=m[0,1]/m[1,1]
        S12=np.linalg.det(m)/m[1,1]
        S21=1/m[1,1]
        S22=-m[1,0]/m[1,1]
        S_list.append([frequency,np.matrix([[S11,S12],[S21,S22]])])
    return S_list

def SFrequencyList_to_ZFrequencyList(s_frequency_list,Z01=complex(50,0),Z02=complex(50,0)):
    """ Converts s parameters into z-parameters. S-parameters should be in the form [[f,S11,S12,S21,S22],...]
    the port 1 (Z01) and port 2 (Z01) impedances can be specified, default is 50. Returns data in the form
    [[f,Z11,Z12,Z21,Z22],...]
    """
    z_frequency_list=[]
    for row in s_frequency_list[:]:
        [frequency,S11,S12,S21,S22]=row
        denominator=-1*((1-S11)*(1-S22)-S12*S21)
        Z11=((Z01.conjugate()+S11*Z01)*(1-S22)+S12*S21*Z01)/denominator
        Z12=(2*S12*(Z01.real*Z02.real)**(.5))/denominator
        Z21=(2*S21*(Z01.real*Z02.real)**(.5))/denominator
        Z22=((1-S11)*(Z02.conjugate()+S22*Z02+S21*S12*Z02))/denominator
        z_frequency_list.append([frequency,Z11,Z12,Z21,Z22])
    return z_frequency_list

def SFrequencyList_to_ABCDFrequencyList(s_frequency_list,Z01=complex(50,0),Z02=complex(50,0)):
    """ Converts s parameters into ABCD-parameters. S-parameters should be in the form [[f,S11,S12,S21,S22],...]
    the port 1 (Z01) and port 2 (Z01) impedances can be specified, default is 50. Returns data in the form
    [[f,A,B,C,D],...]
    """
    ABCD_frequency_list=[]
    for row in s_frequency_list[:]:
        [frequency,S11,S12,S21,S22]=row
        denominator=2*S21*(Z01.real*Z02.real)**(.5)
        A= ((Z01.conjugate()+S11*Z01)*(1-S22)+S12*S21*Z01)/denominator
        B=((Z01.conjugate()+S11*Z01)*(Z02.conjugate()+S22*Z02)-S12*S21*Z01*Z02)/denominator
        C=((1-S11)*(1-S22)-S12*S21)/denominator
        D=((1-S11)*(Z02.conjugate()+S22*Z02)+S12*S21*Z02)/denominator
        ABCD_frequency_list.append([frequency,A,B,C,D])
    return ABCD_frequency_list

def ZFrequencyList_to_TFrequencyList(Z_frequency_list,Z01=complex(50,0),Z02=complex(50,0)):
    """ Converts z parameters into T parameters. Z-parameters should be in the form [[f,Z11,Z12,Z21,Z22],...]
    the port 1 (Z01) and port 2 (Z01) impedances can be specified, default is 50. Returns data in the form
    [[f,T11,T12,T21,T22],...]
    """  
    t_frequency_list=[]
    for row in Z_frequency_list[:]:
        [frequency,Z11,Z12,Z21,Z22]=row
        denominator=2*Z21*(Z01.real*Z02.real)**(.5)
        T11= ((Z11+Z01)*(Z22+Z02)-Z12*Z21)/denominator
        T12=((Z11+Z01)*(Z02.conjugate()-Z22)+Z12*Z21)/denominator
        T21=((Z11-Z01.conjugate())*(Z22+Z02)-Z12*Z21)/denominator
        T22=((Z01.conjugate()-Z11)*(Z22-Z02.conjugate())+Z12*Z21)/denominator
        t_frequency_list.append([frequency,T11,T12,T21,T22])
    return t_frequency_list    

def ABCDFrequencyList_to_TFrequencyList(ABCD_frequency_list,Z01=complex(50,0),Z02=complex(50,0)):
    """ Converts ABCD parameters into T parameters. ABCD-parameters should be in the form [[f,A,B,C,D],...]
    the port 1 (Z01) and port 2 (Z01) impedances can be specified, default is 50. Returns data in the form
    [[f,T11,T12,T21,T22],...]
    """  
    t_frequency_list=[]
    for row in ABCD_frequency_list[:]:
        [frequency,A,B,C,D]=row
        denominator=2*(Z01.real*Z02.real)**(.5)
        T11= (A*Z02+B+C*Z01*Z02+D*Z01)/denominator
        T12=(A*Z02.conjugate()-B+B*Z01*Z02-D*Z01)/denominator
        T21=(A*Z02+B+C*Z01.conjugate()*Z02+D*Z01)/denominator
        T22=(A*Z02.conjugate()-B-C*Z01.conjugate()*Z02.conjugate()+D*Z01.conjugate())/denominator
        t_frequency_list.append([frequency,T11,T12,T21,T22])
    return t_frequency_list

def ABCDFrequencyList_to_YFrequencyList(ABCD_frequency_list):
    """ Converts ABCD parameters into Y-parameters. ABCD-parameters should be in the form [[f,A,B,C,D],...]
    Returns data in the form
    [[f,Y11,Y12,Y21,Y22],...]
    """  
    y_frequency_list=[]
    for row in ABCD_frequency_list[:]:
        [frequency,A,B,C,D]=row
        Y11=D/B
        Y12=(B*C-A*D)/B
        Y21=-1/B
        Y22=A/B
        y_frequency_list.append([frequency,Y11,Y12,Y21,Y22])
    return y_frequency_list 

def YFrequencyList_to_HFrequencyList(y_frequency_list):
    """ Converts Y parameters into h-parameters. ABCD-parameters should be in the form [[f,Y11,Y12,Y21,Y22],...]
    Returns data in the form
    [[f,h11,h12,h21,h22],...]
    """  
    h_frequency_list=[]
    for row in y_frequency_list[:]:
        [frequency,Y11,Y12,Y21,Y22]=row
        h11=1/Y11
        h12=-1*Y12/Y11
        h21=Y21/Y11
        h22=(Y11*Y22-Y12*Y21)/Y11
        h_frequency_list.append([frequency,h11,h12,h21,h22])
    return h_frequency_list 

def ABCDFrequencyList_to_HFrequencyList(ABCD_frequency_list):
    """ Converts ABCD parameters into h-parameters. ABCD-parameters should be in the form [[f,A,B,C,D],...]
    Returns data in the form
    [[f,h11,h12,h21,h22],...]
    """  
    h_frequency_list=[]
    for row in ABCD_frequency_list[:]:
        [frequency,A,B,C,D]=row
        h11=D/B
        h12=(A*D-B*C)/D
        h21=-1/D
        h22=C/D
        h_frequency_list.append([frequency,h11,h12,h21,h22])
    return h_frequency_list    

def HFrequencyList_to_ZFrequencyList(h_frequency_list):
    """ Converts h parameters into Z-parameters. h-parameters should be in the form [[f,h11,h12,h21,h22],...]
    Returns data in the form
    [[f,Z11,Z12,Z21,Z22],...]
    """  
    z_frequency_list=[]
    for row in h_frequency_list[:]:
        [frequency,h11,h12,h21,h22]=row
        Z11=(h11*h22-h12*h21)/h22
        Z12=h12/h22
        Z21=-1*h21/h22
        Z22=1/h22
        z_frequency_list.append([frequency,h11,h12,h21,h22])
    return z_frequency_list 

## Creating a content graph for the parameters
1. Begin with SParameterFrequencyList <-> SParameterFrequencyMatrixList
2. Add SParameterFrequencyMatrixList <-> TParameterFrequencyMatrixList
3. Add TParameterFrequencyMatrixList <-> TParameterFrequencyList
4. Add SParameterFrequencyList -> ZParameterFrequencyList and ZParameterFrequencyList -> TParameterFrequencyList
5. Add ZParameterFrequencyList <-> ZParameterFrequencyMatrixList
6. Add SParameterFrequencyList -> ABCDParameterFrequencyList and ABCDParameterFrequencyList -> TParameterFrequencyList
7. Add ABCDParameterFrequencyList <-> ABCDParameterFrequencyMatrixList
8. Add ABCDParameterFrequencyList -> HParameterFrequencyList and HParameterFrequencyList -> ZParameterFrequencyList
9. Add HParameterFrequencyList <-> HParameterFrequencyMatrixList
10. Add ABCDParameterFrequencyList -> YParameterFrequencyList and YParameterFrequencyList -> HParameterFrequencyList
11. Add YParameterFrequencyList <-> YParameterFrequencyMatrixList




In [135]:
# We begin with the basic graph setup 
graph_options={"graph_name":"Two Port Parameter Graph",
                  "node_names":["SFrequencyList",'SFrequencyMatrixList'],
                  "node_descriptions":["S Parameters","S Parameters in a Matrix"],
                  "current_node":'SFrequencyList',
                  "state":[1,0],
                  "data":[[1.0,0.0001,.999,.999,.0001]],
                  "edge_2_to_1":FrequencyMatrixList_to_FrequencyList,
                  "edge_1_to_2":FrequencyList_to_FrequencyMatrixList,
                  "frequency_units":"GHz",
                  "Z01":50,
                  "Z02":50 }
two_port_parameter_graph=Graph(**graph_options)

In [28]:
two_port_parameter_graph.show()

In [136]:
# now we add the nodes
two_port_parameter_graph.add_node("TFrequencyMatrixList",
                                  "SFrequencyMatrixList",SFrequencyMatrixList_to_TFrequencyMatrixList,
                                  "SFrequencyMatrixList",TFrequencyMatrixList_to_SFrequencyMatrixList,
                                  "T Parameters in a Matrix")

two_port_parameter_graph.add_node("TFrequencyList",
                                  "TFrequencyMatrixList",FrequencyMatrixList_to_FrequencyList,
                                  "TFrequencyMatrixList",FrequencyList_to_FrequencyMatrixList,
                                  "T Parameters")

two_port_parameter_graph.add_node("ZFrequencyList",
                                  "SFrequencyList",SFrequencyList_to_ZFrequencyList,
                                  "TFrequencyList",ZFrequencyList_to_TFrequencyList,
                                  "Z Parameters")

two_port_parameter_graph.add_node("ZFrequencyMatrixList",
                                  "ZFrequencyList",FrequencyList_to_FrequencyMatrixList,
                                  "ZFrequencyList",FrequencyMatrixList_to_FrequencyList,
                                  "Z Parameters in a matrix")

two_port_parameter_graph.add_node("ABCDFrequencyList",
                                  "SFrequencyList",SFrequencyList_to_ABCDFrequencyList,
                                  "TFrequencyList",ABCDFrequencyList_to_TFrequencyList,
                                  "ABCD Parameters")

two_port_parameter_graph.add_node("ABCDFrequencyMatrixList",
                                  "ABCDFrequencyList",FrequencyList_to_FrequencyMatrixList,
                                  "ABCDFrequencyList",FrequencyMatrixList_to_FrequencyList,
                                  "ABCD Parameters in a matrix")

two_port_parameter_graph.add_node("HFrequencyList",
                                  "ABCDFrequencyList",ABCDFrequencyList_to_HFrequencyList,
                                  "ZFrequencyList",HFrequencyList_to_ZFrequencyList,
                                  "h Parameters")

two_port_parameter_graph.add_node("HFrequencyMatrixList",
                                  "HFrequencyList",FrequencyList_to_FrequencyMatrixList,
                                  "HFrequencyList",FrequencyMatrixList_to_FrequencyList,
                                  "H Parameters in a matrix")

two_port_parameter_graph.add_node("YFrequencyList",
                                  "ABCDFrequencyList",ABCDFrequencyList_to_YFrequencyList,
                                  "HFrequencyList",YFrequencyList_to_HFrequencyList,
                                  "Y Parameters")

two_port_parameter_graph.add_node("YFrequencyMatrixList",
                                  "YFrequencyList",FrequencyList_to_FrequencyMatrixList,
                                  "YFrequencyList",FrequencyMatrixList_to_FrequencyList,
                                  "Y Parameters in a matrix")


In [137]:
two_port_parameter_graph.move_to_node("ZFrequencyList")

['edge_SFrequencyList_ZFrequencyList_000']
moving SFrequencyList -> ZFrequencyList


In [35]:
((1)**(.5))

1.0

In [138]:
two_port_parameter_graph.data

[[1.0,
  (-55530.56931312082-0j),
  -55530.5417979875,
  -55530.5417979875,
  (-55527.79556255801-0j)]]

In [139]:
two_port_parameter_graph.move_to_node("TFrequencyList")

['edge_ZFrequencyList_TFrequencyList_000']
moving ZFrequencyList -> TFrequencyList


In [140]:
two_port_parameter_graph.data

[[1.0,
  (1.0267125343619605+0j),
  (-0.02761244427187025-0j),
  (0.0276623942718677+0j),
  (0.9732385066290336-0j)]]

In [133]:
two_port_parameter_graph.move_to_node("SFrequencyList")

['edge_TFrequencyList_TFrequencyMatrixList_000', 'edge_TFrequencyMatrixList_SFrequencyMatrixList_000', 'edge_SFrequencyMatrixList_SFrequencyList_000']
moving TFrequencyList -> TFrequencyMatrixList
moving TFrequencyMatrixList -> SFrequencyMatrixList
moving SFrequencyMatrixList -> SFrequencyList


In [134]:
two_port_parameter_graph.data

[[1.0,
  (0.2001120007205843+0j),
  (0.80000800047957665+0j),
  (0.80000800047990905+0j),
  (0.20007200032066033-0j)]]

In [123]:
two_port_parameter_graph.move_to_node("ABCDFrequencyList")

['edge_SFrequencyList_ABCDFrequencyList_000']
moving SFrequencyList -> ABCDFrequencyList


In [124]:
two_port_parameter_graph.data

[[1.0,
  (0.050000000000000017+0j),
  (999.9999999999684+0j),
  (-0.0010000000000000005+0j),
  (4.4408920985004861e-16+0j)]]

In [125]:
two_port_parameter_graph.move_to_node("SFrequencyList")

equality_list is [False, True, False, True, False, False, False, False, False, False, False, False, False]
equality_list is [False, True, False, True, False, False, False, False, False, False, False, False, False]
equality_list is [False, False, False, False, True, False, True, False, False, False, False, False, False]
equality_list is [False, False, False, False, True, False, True, False, False, False, False, False, False]
equality_list is [False, False, False, False, False, False, False, True, False, True, False, False, False]
equality_list is [False, False, False, False, False, False, False, True, False, True, False, False, False]
['edge_ABCDFrequencyList_YFrequencyList_000', 'edge_YFrequencyList_HFrequencyList_000', 'edge_HFrequencyList_ZFrequencyList_000', 'edge_ZFrequencyList_TFrequencyList_000', 'edge_TFrequencyList_TFrequencyMatrixList_000', 'edge_TFrequencyMatrixList_SFrequencyMatrixList_000', 'edge_SFrequencyMatrixList_SFrequencyList_000']
moving ABCDFrequencyList -> YFrequen

In [126]:
two_port_parameter_graph.data

[[1.0, (1-0j), (-0.0020000000000000634-0j), (-0-0j), (-1.0000050000000003-0j)]]